In [1]:
# Code to be able to import local modules in notebooks
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:

# Import dependencies
# Standard python libraries
import os
import json

# Third party libraries
from oauthlib.oauth2 import WebApplicationClient
import pandas as pd
import numpy as np

# Internal imports
from ffpackage import mfl, analysis, viz
from appmanager import db, user

# Configuration (These variables are stored as environment variables)
GOOGLE_CLIENT_ID = os.environ.get("GOOGLE_CLIENT_ID", None)
GOOGLE_CLIENT_SECRET = os.environ.get("GOOGLE_CLIENT_SECRET", None)
GOOGLE_DISCOVERY_URL = ("https://accounts.google.com/.well-known/openid-configuration")


# Working variables
user_league = "53906"
user_franchise = "0005"
current_week = 15

In [20]:
#def getFranchise():

#franchises = mfl.get_franchises(user_league)['franchiseName'].to_list()
franchises = mfl.get_franchises(user_league)['franchiseName'].to_json()

franchises

'{"0":"Providence Monastic Spirits","1":"Washington Fantasy Football Team","2":"Verdansk Vaqueros FFC","3":"Down with the Sickness unto Death","4":"Crocodilopolis Body Snatchers","5":"O\'Henry\'s Stiffies","6":"Comeback Kings","7":"WHAM! BAM! Thank You CAM!","8":"Pretty Big Wieners","9":"Saskatoon Squatches","10":"Gus The Bus","11":"IDK much about soccer "}'

In [9]:
#def allPlayers():
player_df = db.read_db("predictions")
tables = [player_df.to_html(classes='data')]
titles=player_df.columns.values
#return render_template("allPlayers.html", tables=tables, titles=titles)

,id_mfl,season,week,team,playerName,age,sharkRank,adp,KR,PR,...,defSack,defI,defSaf,defFum,defBlk,defT,defPtsAgainst,defPassYAgainst,defRushYAgainst,defYdsAgainst
0,5848,2022,1.0,TBB,TOM BRADY,45.0,224.0,3000,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,5848,2022,2.0,TBB,TOM BRADY,45.0,224.0,3000,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,5848,2022,6.0,TBB,TOM BRADY,45.0,224.0,3000,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,5848,2022,7.0,TBB,TOM BRADY,45.0,224.0,3000,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,5848,2022,12.0,TBB,TOM BRADY,45.0,224.0,3000,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11980,0532,2022,9.0,HOU,HOUSTON TEXANS,0.0,976.0,3000,0,0,...,2.097930,0.718137,0.018943,0.618927,0.086231,0.182348,25.232143,245.479814,119.651413,365.131228
11981,0532,2022,11.0,HOU,HOUSTON TEXANS,0.0,976.0,3000,0,0,...,2.147516,0.715120,0.024205,0.604407,0.103386,0.171852,25.151553,259.279859,117.695632,376.975492
11982,0532,2022,13.0,HOU,HOUSTON TEXANS,0.0,976.0,3000,0,0,...,2.139411,0.741880,0.021997,0.592463,0.082383,0.172950,24.605641,240.479470,116.555410,357.034879
11983,0532,2022,15.0,HOU,HOUSTON TEXANS,0.0,976.0,3000,0,0,...,2.163755,0.806891,0.022691,0.604643,0.077259,0.180856,24.288947,243.427805,112.743562,356.171366


In [3]:
#@app.route('/predictive', methods=['GET'])
#@login_required
#def predictive():


# Get Franchises in the league
franchise_df = mfl.get_franchises(user_league)
# Append a row to carry free agents
freeAgentRow =  {"franchiseID":["FA"], "franchiseName":["Free Agent"], "franchiseAbbrev":["FA"]}
freeAgentRow = pd.DataFrame.from_dict(freeAgentRow)
franchise_df = pd.concat([franchise_df, freeAgentRow], axis=0, ignore_index=True)

# Get franchise rosters
rosters_df = mfl.get_rosters(user_league)

# Get Free Agents
freeAgent_df = mfl.get_freeAgents(user_league)

# Combine Franchise rosters with free agents to get all players
rosters_df = pd.concat([rosters_df, freeAgent_df], axis=0)

# Get all players, sharkRank, and ADP
player_df = db.read_db("predictions")

# Merge all dfs
complete = player_df.merge(rosters_df, on=['id_mfl', 'week'], how='left').merge(franchise_df, on='franchiseID', how='left')
complete = complete.loc[complete['franchiseID'].notna()]

## Calculate fantasy scores customized based on league-specific scoring rules
scoringDict = {
    'passA': {"multiplier":0, "bins":[-np.inf, np.inf], "labels":[0]},
    'passC': {"multiplier":0, "bins":[-np.inf, np.inf], "labels":[0]},
    'passY': {"multiplier":0.04, "bins":[-np.inf, np.inf], "labels":[0]},
    'passT': {"multiplier":4, "bins":[-np.inf, np.inf], "labels":[0]},
    'passI': {"multiplier":-2, "bins":[-np.inf, np.inf], "labels":[0]},
    'pass2': {"multiplier":2, "bins":[-np.inf, np.inf], "labels":[0]},
    'rushA': {"multiplier":0.1, "bins":[-np.inf, np.inf], "labels":[0]},
    'rushY': {"multiplier":0.1, "bins":[-np.inf, np.inf], "labels":[0]},
    'rushT': {"multiplier":6, "bins":[-np.inf, np.inf], "labels":[0]},
    'rush2': {"multiplier":2, "bins":[-np.inf, np.inf], "labels":[0]},
    'recC': {"multiplier":0.25, "bins":[-np.inf, np.inf], "labels":[0]},
    'recY': {"multiplier":0.1, "bins":[-np.inf, np.inf], "labels":[0]},
    'recT': {"multiplier":6, "bins":[-np.inf, np.inf], "labels":[0]},
    'rec2': {"multiplier":2, "bins":[-np.inf, np.inf], "labels":[0]},
    'fum': {"multiplier":-2, "bins":[-np.inf, np.inf], "labels":[0]},
    'XPA': {"multiplier":0, "bins":[-np.inf, np.inf], "labels":[0]},
    'XPM': {"multiplier":3, "bins":[-np.inf, np.inf], "labels":[0]},
    'FGA': {"multiplier":0, "bins":[-np.inf, np.inf], "labels":[0]},
    'FGM': {"multiplier":3, "bins":[-np.inf, np.inf], "labels":[0]},
    'FG50': {"multiplier":5, "bins":[-np.inf, np.inf], "labels":[0]},
    'defSack': {"multiplier":1, "bins":[-np.inf, np.inf], "labels":[0]},
    'defI': {"multiplier":2, "bins":[-np.inf, np.inf], "labels":[0]},
    'defSaf': {"multiplier":2, "bins":[-np.inf, np.inf], "labels":[0]},
    'defFum': {"multiplier":2, "bins":[-np.inf, np.inf], "labels":[0]},
    'defBlk': {"multiplier":1.5, "bins":[-np.inf, np.inf], "labels":[0]},
    'defT': {"multiplier":6, "bins":[-np.inf, np.inf], "labels":[0]},
    'defPtsAgainst': {"multiplier":0, "bins":[-5,0,6,13,17,21,27,34,45,59,99], "labels":[10,8,7,5,3,2,0,-1,-3,-5]},
    'defPassYAgainst': {"multiplier":0, "bins":[-np.inf, np.inf], "labels":[0]},
    'defRushYAgainst': {"multiplier":0, "bins":[-np.inf, np.inf], "labels":[0]},
    'defYdsAgainst': {"multiplier":0, "bins":[0,274,324,375,425,999], "labels":[5,2,0,-2,-5]}
}
# Calculate predicted fantasy scores based on predicted NFL stats and league-specific scoring rules
analyzed = analysis.calculate_scoresFF(complete, scoringDict)

# Calculate relative values
# Get league-specific info on available slots
posMax = {"QB":2, "RB":5, "WR":6, "TE":5, "PK":2, "DF":2}
posMin = {"QB":1, "RB":2, "WR":2, "TE":2, "PK":2, "DF":2}
totalStarters = 15
predMethod = "scoreTotal"

# Select starters
analyzed = analysis.starterSelector(analyzed, how=predMethod, startersMax=totalStarters, posMax=posMax, posMin=posMin)

# Find the lowest scoring player on the field and set them as the low bar
for x in ["QB", "RB", "WR", "TE", "PK", "DF"]:
    positionMinPred = analyzed.loc[(analyzed['pos']==x) & (analyzed['startSelector']=='Starter'), predMethod].min()
    # Calculate relative values
    analyzed.loc[analyzed['pos']==x, 'relativeValue'] = analyzed.loc[analyzed['pos']==x, predMethod] - positionMinPred

# Convert to json
predictiveData = analyzed.to_json()
#return render_template("predictive.html", predictiveData=predictiveData)

/Users/travisharry/Desktop/Coding/ff_demo2/ffpackage/analysis.py:88: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  scoresBonus.loc[:, scoringDict.keys()] = scoresBonus.loc[:, scoringDict.keys()].astype('float64')


In [12]:
# #@app.route('/compareFranchises')
# #@login_required
# #def compareFranchises():

# # Get Franchises in the league
# franchise_df = mfl.get_franchises(user_league)
# # Append a row to carry free agents
# freeAgentRow =  {"franchiseID":["FA"], "franchiseName":["Free Agent"], "franchiseAbbrev":["FA"]}
# freeAgentRow = pd.DataFrame.from_dict(freeAgentRow)
# franchise_df = pd.concat([franchise_df, freeAgentRow], axis=0, ignore_index=True)

# # Get franchise rosters
# rosters_df = mfl.get_rosters(user_league)

# # Get Free Agents
# freeAgent_df = mfl.get_freeAgents(user_league)

# # Get all players, sharkRank, and ADP
# predictions = db.read_db("predictions")

# # Merge all dfs
# complete = predictions.merge(rosters_df, on='id_mfl', how='left').merge(franchise_df[['franchiseID', 'franchiseName', 'franchiseAbbrev']], on='franchiseID', how='left')
# complete['franchiseID'].fillna("FA", inplace=True)
# complete['franchiseName'].fillna("Free Agent", inplace=True)
# complete['rosterStatus'].fillna("Free Agent", inplace=True)

# # Get info on available slots from MFL site
# posMax = {"QB":2, "RB":5, "WR":6, "TE":5, "PK":2, "DF":2}
# posMin = {"QB":1, "RB":2, "WR":2, "TE":2, "PK":2, "DF":2}
# totalStarters = 15
# predMethod = "pred"

# # Select starters
# df = analysis.starterSelector(complete, how=predMethod, startersMax=totalStarters, posMax=posMax, posMin=posMin)

# # Find the lowest scoring player on the field and set them as the low bar
# for x in ["QB", "RB", "WR", "TE", "PK", "DF"]:
#     positionMinPred = df.loc[(df['pos']==x) & (df['starting']=='Starter'), predMethod].min()
#     # Calculate relative values
#     df.loc[df['pos']==x, 'relativeValue'] = df.loc[df['pos']==x, predMethod] - positionMinPred

# # Visualize the data
# fig = viz.compareFranchises(df, how=predMethod)
# # Encode in JSON format
# graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
# # Render html template in flask
# #return render_template('compareFranchises.html', graphJSON=graphJSON)

In [3]:
#@app.route('/live')
#@login_required
#def live():

## Calculate fantasy scores customized based on league-specific scoring rules
scoringDict = {
    'passA': {"multiplier":0, "bins":[-np.inf, np.inf], "labels":[0]},
    'passC': {"multiplier":0, "bins":[-np.inf, np.inf], "labels":[0]},
    'passY': {"multiplier":0.04, "bins":[-np.inf, np.inf], "labels":[0]},
    'passT': {"multiplier":4, "bins":[-np.inf, np.inf], "labels":[0]},
    'passI': {"multiplier":-2, "bins":[-np.inf, np.inf], "labels":[0]},
    'pass2': {"multiplier":2, "bins":[-np.inf, np.inf], "labels":[0]},
    'rushA': {"multiplier":0.1, "bins":[-np.inf, np.inf], "labels":[0]},
    'rushY': {"multiplier":0.1, "bins":[-np.inf, np.inf], "labels":[0]},
    'rushT': {"multiplier":6, "bins":[-np.inf, np.inf], "labels":[0]},
    'rush2': {"multiplier":2, "bins":[-np.inf, np.inf], "labels":[0]},
    'recC': {"multiplier":0.25, "bins":[-np.inf, np.inf], "labels":[0]},
    'recY': {"multiplier":0.1, "bins":[-np.inf, np.inf], "labels":[0]},
    'recT': {"multiplier":6, "bins":[-np.inf, np.inf], "labels":[0]},
    'rec2': {"multiplier":2, "bins":[-np.inf, np.inf], "labels":[0]},
    'fum': {"multiplier":-2, "bins":[-np.inf, np.inf], "labels":[0]},
    'XPA': {"multiplier":0, "bins":[-np.inf, np.inf], "labels":[0]},
    'XPM': {"multiplier":3, "bins":[-np.inf, np.inf], "labels":[0]},
    'FGA': {"multiplier":0, "bins":[-np.inf, np.inf], "labels":[0]},
    'FGM': {"multiplier":3, "bins":[-np.inf, np.inf], "labels":[0]},
    'FG50': {"multiplier":5, "bins":[-np.inf, np.inf], "labels":[0]},
    'defSack': {"multiplier":1, "bins":[-np.inf, np.inf], "labels":[0]},
    'defI': {"multiplier":2, "bins":[-np.inf, np.inf], "labels":[0]},
    'defSaf': {"multiplier":2, "bins":[-np.inf, np.inf], "labels":[0]},
    'defFum': {"multiplier":2, "bins":[-np.inf, np.inf], "labels":[0]},
    'defBlk': {"multiplier":1.5, "bins":[-np.inf, np.inf], "labels":[0]},
    'defT': {"multiplier":6, "bins":[-np.inf, np.inf], "labels":[0]},
    'defPtsAgainst': {"multiplier":0, "bins":[-5,0,6,13,17,21,27,34,45,59,99], "labels":[10,8,7,5,3,2,0,-1,-3,-5]},
    'defPassYAgainst': {"multiplier":0, "bins":[-np.inf, np.inf], "labels":[0]},
    'defRushYAgainst': {"multiplier":0, "bins":[-np.inf, np.inf], "labels":[0]},
    'defYdsAgainst': {"multiplier":0, "bins":[0,274,324,375,425,999], "labels":[5,2,0,-2,-5]}
}

# Get MFL scoring data
liveScores = mfl.get_liveScoring(user_league)
# Get Franchises in the league
franchises = mfl.get_franchises(user_league)
# Get all players and predictions
predictions = db.read_db("predictions")

# Calculate predicted fantasy scores based on predicted NFL stats and league-specific scoring rules
predictions = analysis.calculate_scoresFF(predictions, scoringDict)
# Filter:
predictions = predictions.loc[predictions['week']==current_week]



# Merge: merge liveScores, franchises, and predictions
df = liveScores.merge(franchises, how='left', on='franchiseID').merge(predictions, how='left', on='id_mfl')

# Clean: convert to float data types
df['liveScore'] = df['liveScore'].astype('float64')
df['secondsRemaining'] = df['secondsRemaining'].astype('float64')

# Filter:
df = df.loc[df['status']=="starter"]

# Analyze:
# # Divide the player's annual prediction by the number of weeks (17) to get a weekly prediction
# df['weeklyPred'] = df['scoreTotal'] / 17
# Calculate each player's expected score at the end of the game
df['expectedLiveScore'] = df.apply(analysis.expectedLiveScore, axis=1)
# Set colors for chart 
df['color'] = df.apply(analysis.colorPicker, axis=1)

# Convert to JSON
liveData = df.to_json()

# # Visualize:
# import plotly
# fig = viz.liveScoring(df)
# # Encode in JSON format
# graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)

# Render html template in flask
#return render_template('live.html', graphJSON=graphJSON)

/Users/travisharry/Desktop/Coding/ff_demo2/ffpackage/analysis.py:88: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  scoresBonus.loc[:, scoringDict.keys()] = scoresBonus.loc[:, scoringDict.keys()].astype('float64')


In [7]:
fig.show()